In [19]:
import sqlite3 
from sqlite3 import Error 
import pandas as pd

In [20]:
def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

In [21]:
connection = create_connection("SQL_booking_3.db")

Connection to SQLite DB successful


In [22]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")

In [23]:
transaction_bd = """
CREATE TABLE IF NOT EXISTS transaction_bd (
  TX_DATETIME NUMERIC NOT NULL,
  CUSTOMER_ID INTEGER,
  TX_AMOUNT REAL
);
"""

In [24]:
execute_query(connection, transaction_bd)

Query executed successfully


In [25]:
df = pd.read_csv('final_transactions.csv')

In [43]:
df.head()

,TX_DATETIME,CUSTOMER_ID,TX_AMOUNT
0,2023-01-01 00:00:31,596,533.07
1,2023-01-01 00:02:10,4961,808.56
2,2023-01-01 00:07:56,2,1442.94
3,2023-01-01 00:09:29,4128,620.65
4,2023-01-01 00:10:34,927,490.66


In [27]:
df.drop(columns = ['TRANSACTION_ID', 'TERMINAL_ID'], inplace = True)

In [28]:
df.to_sql('transaction_bd', connection, if_exists='replace', index=False)

1048575

## A

In [29]:
data_1 = pd.read_sql('SELECT CUSTOMER_ID FROM transaction_bd GROUP BY CUSTOMER_ID HAVING sum(TX_AMOUNT) > 700000 ORDER BY CUSTOMER_ID asc', connection)
data_1

,CUSTOMER_ID
0,389
1,2249
2,2891
3,3116
4,4163


## B

In [ ]:
data_2 = pd.read_sql('SELECT CUSTOMER_ID, SUM(TX_AMOUNT) as TX_SUM FROM transaction_bd GROUP BY CUSTOMER_ID HAVING sum(TX_AMOUNT) > 200000 and TX_DATETIME between "2023-01-01" and "2023-01-13"  ORDER BY CUSTOMER_ID asc', connection)
data_2

## C

In [ ]:
data_3 = pd.read_sql('SELECT CUSTOMER_ID, COUNT(transaction_id) as tx_cnt FROM transaction_bd WHERE SUBSTRING(CUSTOMER_ID, 1, 1) = "4" GROUP BY CUSTOMER_ID HAVING sum(TX_AMOUNT) > 444', connection)
data_3

## D

In [41]:
data_4 = pd.read_sql('SELECT TX_DATETIME,CUSTOMER_ID,TX_AMOUNT, CASE WHEN sum(TX_AMOUNT)<=50000 THEN "низкая доходность" WHEN sum(TX_AMOUNT)>50000 and sum(TX_AMOUNT)<100000 THEN "средняя доходность" ELSE "высокая доходность" END PROFIT FROM transaction_bd GROUP BY CUSTOMER_ID ORDER BY CUSTOMER_ID asc' , connection)
data_4

,TX_DATETIME,CUSTOMER_ID,TX_AMOUNT,PROFIT
0,2023-01-01 07:19:05,0,1200.41,высокая доходность
1,2023-01-01 05:51:21,1,76.08,высокая доходность
2,2023-01-01 00:07:56,2,1442.94,высокая доходность
3,2023-01-02 17:26:51,3,76.70,низкая доходность
4,2023-01-01 05:34:23,4,916.03,высокая доходность
...,...,...,...,...
4981,2023-01-01 02:09:24,4995,863.29,высокая доходность
4982,2023-01-01 07:28:52,4996,130.16,низкая доходность
4983,2023-01-02 06:29:26,4997,899.90,высокая доходность
4984,2023-01-01 13:02:10,4998,812.16,высокая доходность


## E

In [54]:
data_5 = pd.read_sql('WITH TAB as (SELECT TX_DATETIME,CUSTOMER_ID,TX_AMOUNT, CASE WHEN sum(TX_AMOUNT)<=50000 THEN "низкая доходность" WHEN sum(TX_AMOUNT)>50000 and sum(TX_AMOUNT)<100000 THEN "средняя доходность" ELSE "высокая доходность" END PROFIT FROM transaction_bd GROUP BY CUSTOMER_ID ORDER BY CUSTOMER_ID asc) SELECT TAB.PROFIT, count(TAB.PROFIT) as QUANTITY_CUSTOMER_BY_PROFIT FROM TAB GROUP BY PROFIT', connection)
data_5

,PROFIT,QUANTITY_CUSTOMER_BY_PROFIT
0,высокая доходность,2167
1,низкая доходность,1812
2,средняя доходность,1007


## F

In [59]:
data_6 = pd.read_sql('SELECT SUBSTRING (TX_DATETIME, 1, 10) as DATE, sum(TX_AMOUNT) as SUM_BY_DAY FROM transaction_bd GROUP BY DATE ORDER BY DATE asc', connection)
data_6

,DATE,SUM_BY_DAY
0,2023-01-01,4827656.26
1,2023-01-02,4862551.41
2,2023-01-03,5058973.71
3,2023-01-04,4938142.47
4,2023-01-05,5002954.23
...,...,...
105,2023-04-16,5299386.81
106,2023-04-17,5100973.31
107,2023-04-18,5233557.39
108,2023-04-19,5194846.44


In [60]:
connection.close()

## Подготовка дашборда

In [63]:
!pip install dash
!pip install jupyter-dash

In [ ]:
from dash import Dash, html, dcc
import plotly.express as px

app = Dash()

fig1 = px.bar(data_6, x="DATE", y="SUM_BY_DAY", title = 'Cумма транзакций за день')

fig2 = px.histogram(data_6, x="SUM_BY_DAY" , title='Распределение сумм транзакций за день', nbins=50)

app.layout = html.Div(children=[
    html.H1(children='Данные по совершенным транзакциям'),

    html.Div(children='Сумма транзакций за каждый день'),

    dcc.Graph(
        id='example-graph',
        figure=fig1
    ),
     dcc.Graph(
        id='example-graph1',
        figure=fig2
    )
])

app.run_server()